In [ ]:
# -*- coding: utf-8 -*-
"""
NeuroDoc Data Generator — Генерация корпоративной документации через Sber GigaChat API
Автор: Дмитрий Блюхеров
Цель: создать структуру data/docs/{hr,it,onboarding,compliance} с реалистичными документами,
сгенерированными через российскую LLM (GigaChat) через официальный GigaChat API
"""

# === УСТАНОВКА ЗАВИСИМОСТЕЙ ===
!pip install aiohttp python-dotenv requests -q
!pip install pdf2docx pandoc -q

import aiohttp
import subprocess
from pdf2docx import Converter
import os
import asyncio
import json
import uuid
from pathlib import Path
from google.colab import userdata, drive
import requests
from datetime import datetime

In [24]:
# === НАСТРОЙКА GOOGLE DRIVE ===
print("Подключаем Google Диск...")
drive.mount('/content/drive')

# Папка для проекта на Диске
PROJECT_DRIVE_PATH = Path("/content/drive/MyDrive/NeuroDoc_Data")
PROJECT_DRIVE_PATH.mkdir(parents=True, exist_ok=True)

# Локальная рабочая директория (временная)
LOCAL_DATA_PATH = Path("/content/neurodoc_data")
LOCAL_DATA_PATH.mkdir(parents=True, exist_ok=True)

Подключаем Google Диск...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === ПОЛУЧЕНИЕ СЕКРЕТОВ ИЗ COLAB ===
print("\n" + "="*70)
print("🔐 ЗАГРУЗКА СЕКРЕТОВ ИЗ COLAB")
print("="*70)
try:
    # Получаем Authorization Key (Base64 от Client ID:Client Secret)
    # userdata.get() принимает только один аргумент (имя ключа)
    GIGACHAT_AUTH_KEY = userdata.get("GIGACHAT_AUTH_KEY")
    
    # Получаем Scope (если не задан, используем значение по умолчанию)
    GIGACHAT_SCOPE_RAW = userdata.get("GIGACHAT_SCOPE")
    if GIGACHAT_SCOPE_RAW:
        GIGACHAT_SCOPE = GIGACHAT_SCOPE_RAW
    else:
        GIGACHAT_SCOPE = "GIGACHAT_API_PERS"  # Значение по умолчанию
    
    if not GIGACHAT_AUTH_KEY:
        raise ValueError("GIGACHAT_AUTH_KEY не задан")
    
    print(f"✅ Authorization Key загружен (длина: {len(GIGACHAT_AUTH_KEY)} символов)")
    print(f"✅ Scope: {GIGACHAT_SCOPE}")
    print(f"📝 Первые 20 символов ключа: {GIGACHAT_AUTH_KEY[:20]}...")
    print(f"📝 Последние 10 символов ключа: ...{GIGACHAT_AUTH_KEY[-10:]}")
    print("\n" + "="*70)
    
except Exception as e:
    print(f"❌ Ошибка: {e}")
    print("\n❗ Перейдите в Colab → Runtime → Secrets и добавьте:")
    print("   - GIGACHAT_AUTH_KEY (Base64 от Client ID:Client Secret)")
    print("   - GIGACHAT_SCOPE (опционально, по умолчанию GIGACHAT_API_PERS)")
    print("\n💡 Как получить Authorization Key:")
    print("   1. В личном кабинете GigaChat получите Client ID и Client Secret")
    print("   2. Закодируйте их в Base64: base64(Client ID:Client Secret)")
    print("   3. Добавьте результат в секреты Colab как GIGACHAT_AUTH_KEY")
    print("\n💡 Возможные значения Scope:")
    print("   - GIGACHAT_API_PERS (для физических лиц)")
    print("   - GIGACHAT_API_B2B (для ИП и юридических лиц по платным пакетам)")
    print("   - GIGACHAT_API_CORP (для ИП и юридических лиц по схеме pay-as-you-go)")
    raise SystemExit()


Загружаем секреты из Colab...
✅ Секреты загружены успешно


In [26]:
# === 1. СКАЧИВАНИЕ ЗАКОНОВ РФ ===

# Папка для результатов
compliance_dir = Path("/content/neurodoc_data/compliance")
compliance_dir.mkdir(parents=True, exist_ok=True)

# Надёжные прямые ссылки на PDF (Garant)
laws = {
    "tckrf": "https://www.fintech.ru/uploads/Трудовой%20кодекс%20РФ.pdf",  # если уже работает — оставляем
    "gckrf": "https://www.garant.ru/files/9/2/343529/garant_grajdansky_kodeks_rf.pdf",
    "uckrf": "https://www.garant.ru/download/?/files/9/2/343529/garant_ugolovny_kodeks_rf.pdf",
}

def download_file(url: str, dst: Path):
    print(f"🔽 Скачивание: {url}")
    resp = requests.get(url, stream=True, timeout=30)
    resp.raise_for_status()
    with open(dst, "wb") as f:
        for chunk in resp.iter_content(1024*32):
            if chunk:
                f.write(chunk)

def pdf_to_md(pdf_path: str, md_path: str):
    # PDF -> DOCX (pdf2docx)
    docx_path = pdf_path.replace(".pdf", ".docx")
    try:
        conv = Converter(pdf_path)
        conv.convert(docx_path, start=0, end=None)
        conv.close()
    except Exception as e:
        print(f"⚠️ Ошибка при PDF→DOCX: {e}")
        return False

    # DOCX -> Markdown (pandoc)
    try:
        subprocess.run(["pandoc", docx_path, "-t", "markdown", "-o", md_path], check=True)
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Ошибка при DOCX→MD (pandoc): {e}")
        return False

    return True

for code, url in laws.items():
    pdf_file = compliance_dir / f"{code}.pdf"
    md_file = compliance_dir / f"{code}.md"
    try:
        download_file(url, pdf_file)
    except Exception as e:
        print(f"❌ Ошибка при скачивании {url}: {e}")
        continue

    print(f"🔄 Конвертация {pdf_file.name} → {md_file.name}")
    ok = pdf_to_md(str(pdf_file), str(md_file))
    if ok:
        # Небольшая постобработка: сделать заголовок и поправить 'Статья'
        try:
            txt = md_file.read_text(encoding="utf-8")
            txt = txt.replace("\r\n", "\n")
            txt = txt.replace("Статья ", "\n## Статья ")
            md_file.write_text(f"# {code.upper()}\n\n" + txt, encoding="utf-8")
        except Exception as e:
            print(f"⚠️ Не удалось улучшить MD (код {code}): {e}")
        print(f"✅ Готово: {md_file}")
    else:
        print(f"❌ Конвертация не удалась для {pdf_file.name}.")

🔽 Скачивание: https://www.fintech.ru/uploads/Трудовой%20кодекс%20РФ.pdf
🔄 Конвертация tckrf.pdf → tckrf.md
✅ Готово: /content/neurodoc_data/compliance/tckrf.md
🔽 Скачивание: https://www.garant.ru/files/9/2/343529/garant_grajdansky_kodeks_rf.pdf
🔄 Конвертация gckrf.pdf → gckrf.md
✅ Готово: /content/neurodoc_data/compliance/gckrf.md
🔽 Скачивание: https://www.garant.ru/download/?/files/9/2/343529/garant_ugolovny_kodeks_rf.pdf
🔄 Конвертация uckrf.pdf → uckrf.md
✅ Готово: /content/neurodoc_data/compliance/uckrf.md


In [ ]:
# === 2. ГЕНЕРАЦИЯ ДОКУМЕНТОВ ЧЕРЕЗ GIGACHAT API ===
print("\n" + "="*70)
print("🔧 НАСТРОЙКА GIGACHAT API")
print("="*70)

# Официальные endpoints согласно документации GigaChat API
OAUTH_TOKEN_URL = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
GIGACHAT_API_URL = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

print(f"📍 OAuth endpoint: {OAUTH_TOKEN_URL}")
print(f"📍 Chat completions endpoint: {GIGACHAT_API_URL}")
print("="*70)

# Глобальная переменная для кэширования токена
_access_token_cache = None
_token_expires_at = 0

async def get_gigachat_token(session=None):
    """Получает токен доступа GigaChat API по схеме OAuth 2.0"""
    global _access_token_cache, _token_expires_at
    
    # Проверяем кэш токена (действителен 30 минут)
    current_time = datetime.now().timestamp()
    if _access_token_cache and current_time < _token_expires_at - 60:  # Обновляем за минуту до истечения
        print(f"♻️  Используем кэшированный токен (истекает через {int((_token_expires_at - current_time) / 60)} мин)")
        return _access_token_cache
    
    print("\n" + "-"*70)
    print("🔑 ПОЛУЧЕНИЕ ТОКЕНА ДОСТУПА GIGACHAT API")
    print("-"*70)
    
    # Генерируем уникальный идентификатор запроса (UUID4)
    rq_uid = str(uuid.uuid4())
    print(f"📋 RqUID: {rq_uid}")
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": rq_uid,
        "Authorization": f"Basic {GIGACHAT_AUTH_KEY}"
    }
    
    data = {
        "scope": GIGACHAT_SCOPE
    }
    
    print(f"📤 Отправка запроса на получение токена...")
    print(f"   Scope: {GIGACHAT_SCOPE}")
    print(f"   Authorization: Basic {GIGACHAT_AUTH_KEY[:20]}...")
    print(f"   ⚠️  Отключена проверка SSL-сертификата (требуется для OAuth endpoint)")
    
    try:
        # Создаем отдельную сессию с отключенной проверкой SSL для OAuth endpoint
        # так как используется самоподписанный сертификат
        import ssl
        
        # Создаем SSL контекст без проверки сертификатов
        ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        
        # Используем ssl_context в коннекторе
        connector = aiohttp.TCPConnector(ssl=ssl_context)
        
        # timeout для запроса
        timeout = aiohttp.ClientTimeout(total=30)
        
        async with aiohttp.ClientSession(connector=connector, timeout=timeout) as oauth_session:
            async with oauth_session.post(OAUTH_TOKEN_URL, headers=headers, data=data) as resp:
                print(f"📥 Статус ответа: {resp.status}")
                print(f"📥 Заголовки ответа: {dict(resp.headers)}")
                
                if resp.status == 200:
                    response_data = await resp.json()
                    print(f"✅ Токен получен успешно!")
                    print(f"📝 Access token (первые 50 символов): {response_data.get('access_token', '')[:50]}...")
                    
                    _access_token_cache = response_data["access_token"]
                    # expires_at в миллисекундах, конвертируем в секунды
                    expires_at_ms = response_data.get("expires_at", 0)
                    _token_expires_at = expires_at_ms / 1000 if expires_at_ms > 1000000000000 else expires_at_ms
                    
                    expires_time = datetime.fromtimestamp(_token_expires_at).strftime("%Y-%m-%d %H:%M:%S")
                    print(f"⏰ Токен действителен до: {expires_time}")
                    print("-"*70)
                    return _access_token_cache
                else:
                    response_text = await resp.text()
                    print(f"❌ Ошибка получения токена (статус {resp.status})")
                    print(f"📄 Ответ сервера: {response_text[:500]}")
                    print("-"*70)
                    return None
                    
    except Exception as e:
        print(f"❌ Исключение при получении токена: {type(e).__name__}: {e}")
        import traceback
        print(f"📋 Traceback: {traceback.format_exc()[:500]}")
        print("-"*70)
        return None

async def generate_with_gigachat(session, prompt, model="GigaChat", temperature=0.7, max_tokens=2000):
    """Генерирует текст через GigaChat API"""
    print("\n" + "-"*70)
    print(f"🤖 ГЕНЕРАЦИЯ ТЕКСТА ЧЕРЕЗ GIGACHAT API")
    print("-"*70)
    
    # Получаем токен доступа
    access_token = await get_gigachat_token(session)
    if not access_token:
        print("❌ Не удалось получить токен доступа")
        return ""
    
    # Генерируем уникальный идентификатор запроса
    request_id = str(uuid.uuid4())
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "X-Request-ID": request_id
    }
    
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": False
    }
    
    print(f"📋 Request ID: {request_id}")
    print(f"🤖 Модель: {model}")
    print(f"🌡️  Temperature: {temperature}")
    print(f"🔢 Max tokens: {max_tokens}")
    print(f"📝 Промпт (первые 100 символов): {prompt[:100]}...")
    print(f"📤 Отправка запроса на генерацию...")
    print(f"   ⚠️  Отключена проверка SSL-сертификата (требуется для API endpoint)")
    
    try:
        # Создаем отдельную сессию с отключенной проверкой SSL для основного API
        import ssl
        
        # Создаем SSL контекст без проверки сертификатов
        ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        
        # Используем ssl_context в коннекторе
        connector = aiohttp.TCPConnector(ssl=ssl_context)
        
        # timeout для запроса
        timeout = aiohttp.ClientTimeout(total=60)
        
        async with aiohttp.ClientSession(connector=connector, timeout=timeout) as api_session:
            async with api_session.post(GIGACHAT_API_URL, json=payload, headers=headers) as resp:
                print(f"📥 Статус ответа: {resp.status}")
                
                if resp.status == 200:
                    response_data = await resp.json()
                    print(f"✅ Генерация успешна!")
                    
                    # Извлекаем текст из ответа согласно формату API
                    if "choices" in response_data and len(response_data["choices"]) > 0:
                        content = response_data["choices"][0]["message"]["content"]
                        
                        # Выводим статистику использования токенов
                        if "usage" in response_data:
                            usage = response_data["usage"]
                            print(f"📊 Использование токенов:")
                            print(f"   - Prompt tokens: {usage.get('prompt_tokens', 0)}")
                            print(f"   - Completion tokens: {usage.get('completion_tokens', 0)}")
                            print(f"   - Total tokens: {usage.get('total_tokens', 0)}")
                            if "precached_prompt_tokens" in usage:
                                print(f"   - Precached tokens: {usage.get('precached_prompt_tokens', 0)}")
                        
                        print(f"📝 Длина сгенерированного текста: {len(content)} символов")
                        print(f"📝 Первые 150 символов: {content[:150]}...")
                        print("-"*70)
                        return content.strip()
                    else:
                        print(f"⚠️  Неожиданный формат ответа: {json.dumps(response_data, ensure_ascii=False, indent=2)[:500]}")
                        print("-"*70)
                        return ""
                else:
                    response_text = await resp.text()
                    print(f"❌ Ошибка генерации (статус {resp.status})")
                    print(f"📄 Ответ сервера: {response_text[:1000]}")
                    print("-"*70)
                    return ""
                    
    except Exception as e:
        print(f"❌ Исключение при генерации: {type(e).__name__}: {e}")
        import traceback
        print(f"📋 Traceback: {traceback.format_exc()[:500]}")
        print("-"*70)
        return ""

# Темы HR
HR_TOPICS = [
    "Политика удалённой работы", "Порядок оформления отпуска", "Больничный лист: правила и компенсации",
    "Корпоративная этика и поведение", "Политика конфиденциальности", "Премии и бонусы",
    "Оценка эффективности (KPI)", "Онбординг нового сотрудника", "Увольнение по инициативе работника",
    "Командировки: правила и возмещение"
] * 5  # 50 документов

# Темы IT
IT_TOPICS = [
    "Инструкция по подключению к корпоративному VPN", "Порядок запроса доступа к базам данных",
    "Инструкция по работе с корпоративной почтой", "Политика использования личных устройств (BYOD)",
    "Инструкция по сбросу пароля в Active Directory", "Порядок установки корпоративного ПО",
    "Инструкция по работе с Jira и Confluence", "Политика резервного копирования данных",
    "Инструкция по защите персональных данных", "Порядок обращения в техподдержку"
] * 5  # 50 документов

# Онбординг
ONBOARDING_TOPICS = [
    "Как начать работу в компании: первый день",
    "Руководство по корпоративным инструментам (почта, Jira, Confluence)",
    "Безопасность и работа с конфиденциальной информацией",
    "Кто ваш контакт в IT и HR",
    "Как подключиться к рабочим ресурсам из дома"
]

async def generate_and_save(session, topic, index, category, prompt_template, model="GigaChat"):
    """Генерирует документ и сохраняет его в файл"""
    print(f"\n{'='*70}")
    print(f"📄 [{category.upper()}] Документ {index+1}: {topic}")
    print(f"{'='*70}")
    
    prompt = prompt_template.format(topic=topic)
    content = await generate_with_gigachat(session, prompt, model=model, temperature=0.7, max_tokens=2000)
    
    if content:
        # Форматируем имя файла
        safe_name = topic.lower().replace(" ", "_").replace("/", "_").replace(".", "").replace(":", "")
        safe_name = "".join(c for c in safe_name if c.isalnum() or c in ("_", "-"))[:100]  # Ограничиваем длину
        local_path = LOCAL_DATA_PATH / category / f"{category}_{index+1:02d}_{safe_name}.md"
        local_path.parent.mkdir(parents=True, exist_ok=True)
        local_path.write_text(f"# {topic}\n\n{content}", encoding="utf-8")
        print(f"💾 Файл сохранён: {local_path}")
        print(f"✅ [{category.upper()}] {index+1:02d}: {topic} - ГОТОВО")
        return True
    else:
        print(f"❌ [{category.upper()}] {index+1:02d}: {topic} - ОШИБКА ГЕНЕРАЦИИ")
        return False

HR_PROMPT = """Напиши официальный HR-документ на тему: «{topic}».
Стиль: деловой, структурированный, с заголовками (Введение, Основная часть, Заключение).
Объём: 300–500 слов.
Пиши на русском языке.
Формат: только Markdown без кодовых блоков."""

IT_PROMPT = """Напиши официальную IT-инструкцию на тему: «{topic}».
Стиль: пошаговый, технический, с нумерованными шагами.
Объём: 300–500 слов.
Пиши на русском языке.
Формат: только Markdown без кодовых блоков."""

ONB_PROMPT = """Напиши онбординг-гайд для новых сотрудников на тему: «{topic}».
Стиль: дружелюбный, но информативный, с чек-листами.
Объём: 250–400 слов.
Пиши на русском языке.
Формат: только Markdown."""


In [ ]:
# === ЗАПУСК ГЕНЕРАЦИИ ===
print("\n" + "="*70)
print("🚀 ЗАПУСК ГЕНЕРАЦИИ ДОКУМЕНТОВ")
print("="*70)

# Создаём директории для категорий
for category in ["hr", "it", "onboarding"]:
    (LOCAL_DATA_PATH / category).mkdir(parents=True, exist_ok=True)
    print(f"📁 Создана директория: {LOCAL_DATA_PATH / category}")

# Генерируем HR-документы
print(f"\n{'='*70}")
print(f"🧠 ГЕНЕРАЦИЯ HR-ДОКУМЕНТОВ ({len(HR_TOPICS)} шт) через GigaChat API")
print(f"{'='*70}")
hr_success = 0
hr_failed = 0

async with aiohttp.ClientSession() as session:
    for i, topic in enumerate(HR_TOPICS, 1):
        try:
            result = await generate_and_save(session, topic, i-1, "hr", HR_PROMPT, model="GigaChat")
            if result:
                hr_success += 1
            else:
                hr_failed += 1
            # Небольшая задержка между запросами, чтобы не превысить лимиты
            await asyncio.sleep(0.5)
        except Exception as e:
            print(f"❌ Критическая ошибка при генерации HR-документа {i}: {e}")
            hr_failed += 1

print(f"\n📊 HR-документы: ✅ {hr_success} успешно, ❌ {hr_failed} ошибок")

# Генерируем IT-инструкции
print(f"\n{'='*70}")
print(f"💻 ГЕНЕРАЦИЯ IT-ИНСТРУКЦИЙ ({len(IT_TOPICS)} шт) через GigaChat API")
print(f"{'='*70}")
it_success = 0
it_failed = 0

async with aiohttp.ClientSession() as session:
    for i, topic in enumerate(IT_TOPICS, 1):
        try:
            result = await generate_and_save(session, topic, i-1, "it", IT_PROMPT, model="GigaChat")
            if result:
                it_success += 1
            else:
                it_failed += 1
            await asyncio.sleep(0.5)
        except Exception as e:
            print(f"❌ Критическая ошибка при генерации IT-инструкции {i}: {e}")
            it_failed += 1

print(f"\n📊 IT-инструкции: ✅ {it_success} успешно, ❌ {it_failed} ошибок")

# Генерируем онбординг-гайды
print(f"\n{'='*70}")
print(f"🆕 ГЕНЕРАЦИЯ ОНБОРДИНГ-ГАЙДОВ ({len(ONBOARDING_TOPICS)} шт) через GigaChat API")
print(f"{'='*70}")
onb_success = 0
onb_failed = 0

async with aiohttp.ClientSession() as session:
    for i, topic in enumerate(ONBOARDING_TOPICS, 1):
        try:
            result = await generate_and_save(session, topic, i-1, "onboarding", ONB_PROMPT, model="GigaChat")
            if result:
                onb_success += 1
            else:
                onb_failed += 1
            await asyncio.sleep(0.5)
        except Exception as e:
            print(f"❌ Критическая ошибка при генерации онбординг-гайда {i}: {e}")
            onb_failed += 1

print(f"\n📊 Онбординг-гайды: ✅ {onb_success} успешно, ❌ {onb_failed} ошибок")

print(f"\n{'='*70}")
print("🎉 ГЕНЕРАЦИЯ ЗАВЕРШЕНА!")
print(f"{'='*70}")


🧠 Генерация HR-документов (50 шт) через GigaChat...
❌ Ошибка запроса к GigaChat: 403, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://gigachat.api.cloud.ru/api/gigachat/v1/chat/completions'
❌ Ошибка запроса к GigaChat: 403, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://gigachat.api.cloud.ru/api/gigachat/v1/chat/completions'
❌ Ошибка запроса к GigaChat: 403, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://gigachat.api.cloud.ru/api/gigachat/v1/chat/completions'
❌ Ошибка запроса к GigaChat: 403, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://gigachat.api.cloud.ru/api/gigachat/v1/chat/completions'
❌ Ошибка запроса к GigaChat: 403, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://gigachat.api.cloud.ru/api/gigachat/v1/chat/completions'
❌ 

In [ ]:
# === КОПИРОВАНИЕ НА GOOGLE DRIVE ===
import shutil
print("\n📤 Сохраняем данные на Google Диск...")
shutil.copytree(LOCAL_DATA_PATH, PROJECT_DRIVE_PATH, dirs_exist_ok=True)
print(f"✅ Все документы сохранены в: {PROJECT_DRIVE_PATH}")

In [ ]:
# === ИТОГ ===
hr_count = len(list((LOCAL_DATA_PATH / "hr").glob("*.md")))
it_count = len(list((LOCAL_DATA_PATH / "it").glob("*.md")))
onb_count = len(list((LOCAL_DATA_PATH / "onboarding").glob("*.md")))
compliance_count = len(list((LOCAL_DATA_PATH / "compliance").glob("*.md")))

print("\n🎉 Генерация завершена!")
print(f"📁 HR-документов: {hr_count}")
print(f"📁 IT-инструкций: {it_count}")
print(f"📁 Онбординг-гайдов: {onb_count}")
print(f"📁 Законов РФ: {compliance_count}")
print(f"\n✅ Данные готовы для использования в RAG-ассистенте на Sber Cloud!")